- Get RAW (not normalised) data
- Scale it using StandardScaler
- Perform PCA
- Run LoF on this new Dataset

In [ ]:
%reset

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #For 3D Plots

#Required by Jupyter for inlining images
%matplotlib inline

Read the Data:

In [4]:
Data = pd.read_pickle('../Data/Beam1_MergedRawData.pkl')

In [5]:
Data.head()

,Time Stamp Abort Gap,Abort Gap Reading,Time Stamp SPS,Time Stamp LHC,LHC-SPS,Time Stamp BLM,BLM,Time Stamp BPM,BPM Reading 1,BPM Reading 2,...,BPM Reading 13,BPM Reading 14,BPM Reading 15,BPM Reading 16,BPM Reading 17,BPM Reading 18,BPM MSE,Time Stamp Bunches,Number of Bunches,Change in Number of Bunches
0,1.534476e+09,5.136406e+08,1.534476e+09,1.534476e+09,-9.910938e+08,1.534476e+09,0.003960,1.534476e+09,-0.062,-0.134,...,-1.127,-1.339,0.204,-1.436,-0.358,-0.765,0.013448,1.534476e+09,109.0,96.0
1,1.534476e+09,1.605127e+08,1.534476e+09,1.534476e+09,1.496010e+09,1.534476e+09,0.002312,1.534476e+09,-0.146,0.082,...,-1.098,-1.444,0.073,-1.414,-0.433,-0.730,0.005628,1.534476e+09,121.0,12.0
2,1.534476e+09,-2.453854e+08,1.534476e+09,1.534476e+09,1.760888e+10,1.534476e+09,0.004158,1.534476e+09,-0.104,-0.037,...,-1.143,-1.347,0.132,-1.396,-0.379,-0.668,0.008905,1.534476e+09,265.0,144.0
3,1.534477e+09,3.852305e+08,1.534477e+09,1.534477e+09,-1.270084e+09,1.534477e+09,0.003327,1.534477e+09,-0.014,-0.008,...,-0.971,-1.447,0.268,-1.489,-0.406,-0.696,0.011799,1.534477e+09,277.0,12.0
4,1.534477e+09,-2.889211e+08,1.534477e+09,1.534477e+09,1.024349e+10,1.534477e+09,0.004684,1.534477e+09,0.008,-0.017,...,-1.130,-1.332,0.253,-1.402,-0.345,-0.696,0.021249,1.534477e+09,421.0,144.0


### Scaling the Data:

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
features = ['Abort Gap Reading','LHC-SPS','BLM','BPM Reading 1','BPM Reading 2','BPM Reading 3','BPM Reading 4',
           'BPM Reading 5','BPM Reading 6','BPM Reading 7','BPM Reading 8','BPM Reading 9','BPM Reading 10',
           'BPM Reading 11','BPM Reading 12','BPM Reading 13','BPM Reading 14','BPM Reading 15','BPM Reading 16',
           'BPM Reading 17','BPM Reading 18','Change in Number of Bunches']
#Separate out the features
x = Data.loc[:,features].values
x = StandardScaler().fit_transform(x)

### PCA:

In [9]:
from sklearn.decomposition import PCA

In [12]:
pca = PCA(n_components=0.95) ##At least 95% of the Variance must be explained

In [13]:
principalComponents = pca.fit_transform(x)
df = pd.DataFrame(data = principalComponents)
df.head()

,0,1,2,3,4,5,6,7,8
0,1.192887,-4.049816,2.403280,-2.310358,-1.764720,-0.308992,-0.872477,0.468545,1.369542
1,2.619941,-2.685088,2.107837,-0.856375,-1.749996,-1.383004,-0.635971,0.404023,1.299116
2,1.414478,-3.291289,2.174845,-1.219189,-2.007727,-0.683955,-0.738607,0.408858,1.305568
3,2.065089,-2.379913,2.598642,-1.942886,-2.259220,-0.134746,-1.062125,0.339277,1.428992
4,0.504328,-2.930783,2.381649,-2.022577,-1.940353,-0.072183,-0.887856,0.302781,1.377509


### LOF:

In [14]:
from sklearn.neighbors import LocalOutlierFactor

In [16]:
lof = LocalOutlierFactor(n_neighbors=20,contamination=0.01, n_jobs=-1).fit_predict(df)

/home/marc/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [17]:
df['Predicted Value'] = lof
is_anomaly = df['Predicted Value']==-1
Anomalies = df[is_anomaly]

not_anomaly = df['Predicted Value']!=-1
Inliers = df[not_anomaly]

In [20]:
Anomalies

,0,1,2,3,4,5,6,7,8,Predicted Value
115,8.108279,2.950846,-0.381687,5.108494,4.978379,1.195623,-0.357596,-0.856778,-0.630759,-1
683,-2.890771,1.651173,3.945290,-2.446962,-1.401144,-3.252363,0.101522,-0.934068,-0.754736,-1
742,4.685409,5.612194,0.185208,3.774192,-2.919272,-2.537396,-1.388012,1.026151,4.077139,-1
743,5.566280,6.839278,0.191277,4.200048,-2.508343,-3.591144,-0.906691,0.657987,3.712989,-1
745,4.179959,6.177619,0.559555,4.948514,-2.920288,-4.775490,-0.631876,0.570077,3.781576,-1
748,4.686945,6.561072,0.772487,3.576420,-3.151190,-3.269786,-0.637340,0.244739,2.339974,-1
816,0.235481,5.474587,9.312584,1.773144,3.098697,6.288993,10.312150,8.499885,0.243587,-1
836,1.991850,4.337076,12.199238,0.438775,2.236952,1.634726,11.319461,7.673709,0.976364,-1
865,1.255215,4.092025,10.568441,1.324907,0.636783,4.560336,10.786360,8.853971,0.654635,-1


In [19]:
len(Anomalies[0])

9

### Use MinMax Scaler

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
x = Data.loc[:,features].values
x = MinMaxScaler().fit_transform(x)

In [23]:
principalComponents = pca.fit_transform(x)
df = pd.DataFrame(data = principalComponents)
df.head()

,0,1,2,3,4,5,6,7
0,0.098303,0.749393,0.424510,-0.318786,-0.248304,-0.004700,-0.168659,0.213865
1,0.332374,0.559591,0.408182,-0.108059,-0.170095,0.195879,-0.134845,0.200512
2,0.142694,0.628499,0.417423,-0.150081,-0.260488,0.077346,-0.150691,0.203842
3,0.292241,0.516864,0.508674,-0.243149,-0.282494,-0.015520,-0.166393,0.211186
4,0.027503,0.555561,0.439577,-0.270580,-0.289767,-0.033197,-0.155930,0.214020


In [24]:
lof = LocalOutlierFactor(n_neighbors=20,contamination=0.01, n_jobs=-1).fit_predict(df)

/home/marc/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [25]:
df['Predicted Value'] = lof
is_anomaly = df['Predicted Value']==-1
Anomalies = df[is_anomaly]

not_anomaly = df['Predicted Value']!=-1
Inliers = df[not_anomaly]

In [26]:
Anomalies

,0,1,2,3,4,5,6,7,Predicted Value
19,0.546369,-0.469928,0.093894,0.574088,-0.142906,0.099929,-0.178534,0.092621,-1
115,1.359251,-0.328325,-0.206570,0.601985,1.009600,-0.189818,0.111288,-0.061066,-1
683,-0.415599,-0.244931,0.670282,-0.489393,-0.089768,0.443940,0.082577,-0.090730,-1
742,0.945810,-0.773328,0.292678,0.655421,-0.199909,0.456188,-0.292677,0.550704,-1
743,1.094754,-0.957473,0.253283,0.676313,-0.112297,0.642393,-0.210382,0.497613,-1
745,0.823178,-0.897470,0.352652,0.761563,-0.112602,0.829835,-0.175855,0.509076,-1
748,0.950350,-0.899489,0.392224,0.584061,-0.204021,0.578580,-0.117112,0.307807,-1
796,0.085748,-0.602622,0.377580,-0.016485,0.170650,-0.364767,0.041178,-0.172991,-1
866,-0.017416,-0.312698,0.549706,-0.081340,-0.375724,-0.065050,-0.022067,-0.073353,-1
